## Proxy

| Field | Content |
| --- | --- |
| Class | Proxy |
| Collaborator | Method Request, Scheduler, Future |
| Responsibility | Defines the active object's interface to clients; Creates Method Requests; Runs in the client's thread |

## Method Request

| Field | Content |
| --- | --- |
| Class | Method Request |
| Collaborator | Servant, Future |
| Responsibility | Represents a method call on the active object; Provides guards to check when the method request becomes runnable |

## Concrete Method Request

| Field | Content |
| --- | --- |
| Class | Concrete Method Request |
| Collaborator | Servant, Future |
| Responsibility | Implements the representation of a specific method call; Implements guards |

## Active Object Collaboration Diagram

```mermaid
classDiagram
  direction LR

  class Client
  class Proxy {
    method_1()
    method_N()
  }
  class Scheduler {
    dispatch()
    insert()
  }
  class ActivationQueue {
    enqueue()
    dequeue()
  }
  class MethodRequest {
    can_run()
    call()
  }
  class ConcreteMethodRequest1
  class ConcreteMethodRequestN
  class Servant {
    method_1()
    method_N()
  }
  class Future

  Client ..> Proxy : invoke
  Proxy "1" -- "1" Scheduler : schedule
  Scheduler "1" -- "1" ActivationQueue : owns
  Proxy ..> Future : create
  Proxy ..> MethodRequest : create
  ActivationQueue "1" --> "*" MethodRequest : maintain
  MethodRequest --> Servant : execute
  MethodRequest --> Future : write to
  Client --> Future : obtain result from
  MethodRequest <|-- ConcreteMethodRequest1 : inherits
  MethodRequest <|-- ConcreteMethodRequestN : inherits
```


## Active Object Sequence Diagram

```mermaid
sequenceDiagram
  participant Client
  participant Proxy
  participant Future
  participant Scheduler
  participant ActivationList
  participant MethodRequest
  participant Servant

  Client->>Proxy: method()
  Proxy->>Future: create
  Proxy->>MethodRequest: create
  Proxy->>Scheduler: insert()
  Scheduler->>ActivationList: insert()
  Scheduler-->>Proxy: MethodRequest
  Proxy-->>Client: Future

  Scheduler->>ActivationList: dispatch()
  ActivationList-->>Scheduler: remove()
  Scheduler->>MethodRequest: can_run()
  MethodRequest->>Servant: call()
  Servant->>Servant: method()
  Servant-->>MethodRequest: Result
  MethodRequest-->>Future: write to future

  Client->>Future: read from future
  Future-->>Client: Result
```


Come si vede qui il client proxy (che gira in un thread) richiama un metodo sincronizzato. Questo metodo sincronizzato:
1. crea un `MethodRequest` (simile a un command)
2. chiama il metodo `insert()`

Il MethodRequest è una classe Facade che verrà poi specializzata dai ConcreteRequestMethod col solito giochetto puntatore padre = new puntatore figlio. Lo scheduler è in un altro thread, è parte del ActiveObject. Monitora col thread ed esegue. Esegue dispacciando la richiesta al servant. 

## Implementazione
Implementiamo un `ActiveObject` per sincronizzare un MQServant che farà semplicemente un'operazione lenta.

### Dipendenze

Poiché stiamo parlando di thread usiamo il supporto C++ al threading.

## Leitmotif: Message Router

The leitmotif for the whole notebook is a message router implemented with the Active Object pattern.

Execution order is now strict and didactic:
1. theory and role mapping
2. dependencies
3. infrastructure classes
4. user classes
5. runnable end-to-end example at the bottom

### Legacy Notes (Preserved)

Your previous draft cells are preserved below as notes and not executed, so they do not break symbol order.

```cpp
#include <iostream>
#include <string>
#include <thread>

```


```cpp
using namespace std::chrono_literals;

```


```cpp

static void log_line(const std::string& s) {
  std::cout << "[" << std::this_thread::get_id() << "] " << s << "\n";
}

```


```cpp

  std::cout << "Hello, World of concurrency\n";

```


```cpp
  const std::string s = "This is a log message from the main thread";
  log_line(s);

```


Qui lancio in un altro thread e mi aspetto un TID diverso.

```cpp

  const std::string s = "This is a log message from thread 1";
  std::thread t1(log_line, s);
  t1.join();


```


### Attore 1: `MQ_Servant`

Questo implementa le funzionalità core da sincronizzare. Qui dovremmo usare la solita tecnica corretta del C++ con la *Rule of 5* ma non lo farò... Andrò diretto al cuore della classe.
Ho 2 predicati che indicano 3 stati:
```mermaid
stateDiagram-v2
    [*] --> Empty

    Empty --> Neither: put() / not full()
    Neither --> Full: put() / becomes full
    Neither --> Neither: put() / still not full

    Full --> Neither: get() / not empty()
    Neither --> Empty: get() / becomes empty
    Neither --> Neither: get() / still not empty

    Empty --> Empty: get() blocked by empty()
    Full --> Full: put() blocked by full()

```

```cpp
class MQ_Servant{
    public:
        MQ_Servant(size_t mq_size) : mq_size_(mq_size) {}
        ~MQ_Servant() = default;
        
    // Message Queue implementation
    void put(const Message& msg);
    Message get();

    // Predicates
    bool is_full() const;
    bool is_empty() const;

    private:
        std::queue<Message> mq_;
        size_t mq_size_;

}

```


### Actor 2: Invocation infrastructure

Qui scriviamo tutto il circo necessario al client per eseguire il ActiveObject methods.

#### Proxy

Interfaccia verso il servant. Se tiserve un metodo, lo trovi nerl proxy. Il metodo del proxy può esser econsiderato una *closure* del metodo del Servant. Nel contesto io ci metto:
1. I parametri del metodo da eseguire
2. binding al metodo del servant (vanno agganciati...)
3. il `future` in cui appoggiare il risultato (verrà interperllato dal client)
4. la scaffalatura di codice che eseguirà il `MethodRequest`

##### Implemento `MQ_Proxy`

Questo non è altro che un `Factory` che costruisce istanze dei method request e li passa allo scehduler.


```cpp
class MQ_Proxy{
    public:
        enum {
            MQ_SIZE = 10
        };
        // I pass size and let proxy own everything!!
        MQ_Proxy(size_t size = MQ_SIZE) : scheduler_(size), servant_(size) {}

        //Schedule <put> to execute on the active object
        void put(const Message& msg) {
            //This is the third line in the graph
            MethodRequest* mr = new Put(servant_, msg);
            //This is the fourth line in the graph
            scheduler_.insert(mr);
        }

        //Return a <Message_Future> as the "future" result of the asynchronous 
        //<get> method call on the active object
        Message_Future get() {
            // Lo uso per metterci il risultato del get
            Message_Future result;
            // Get uses the result which is injected and managed by function
            MethodRequest* mr = new Get(servant_, result);
            // mr is passed as a pointer onto the heap. When I exit the function, 
            // it is owned by scheduler
            scheduler_.insert(mr);
            // Return by copy
            return result;
        }

        bool is_full() const {
            return servant_.is_full();
        }
        bool is_empty() const {
            return servant_.is_empty();
        }

        // Private state holders injected via constructor injection
        private:
            MQ_Scheduler scheduler_;
            MQ_Servant servant_;
}

```


#### Implemento `method request`

Questi sono dei command object [[GoF95](https://up.curiousprogrammer.dev/docs/skills/system-design/design-patterns/gang-of-four/behavioral-patterns/command/)]. Disaccoppio il scheduler dai dettagli interni di come fare la `MQ_Servant.get()`, ci penserà lui. Tipicamente poi provvediamo un `can_run()` guard method per proteggere l'esecuzione ed un `call()` method usato come hook per metterci il metodo eseguito poi dal servant (eg. tramite delegate o lambda).

 Essendo una interfaccia, andrà poi concretizzata, le classi figlio implementano il command tipizzato sulla GET o la PUT.

```cpp
class Method_Request {
public:
    // Evaluate the synchronization constraint.
    virtual bool can_run () const = 0;

    // Execute the method.
    virtual void call () = 0;
};

```


Essendo una interfaccia, andrà poi concretizzata, le classi figlio implementano il command tipizzato sulla GET o la PUT. Ecco un esempio:

```cpp
class Get : public Method_Request {
    public:
        // I took out const otherwise is ill-typed
        // I pass by value a MQ_Servant
        Get(MQ_Servant* rep, Message_Future& f) : servant_(rep), result_(f) {}
        virtual bool can_run() const override {
            //Synchronization constraint: cannot call if the message queue is empty
            //Here we use the MQ_Servant's is_empty() predicate
            return !servant_->is_empty();
        }
        // keyword virtual, hooks the right call at runtime. If I use the trick 
        // of father  pointer child implementation, I need to set the good one, 
        // otherwise mr->call() will not call correct Get::call()
        virtual void call() override{
            // Bind dequeued message to the future result.
            result_ = servant_->get();
        }
    private:
        // I do not own the servant
        MQ_Servant* servant_;
        Message_Future result_;
};

```


```mermaid
sequenceDiagram
actor Client
participant Ref as ref: AbstractClass
participant Obj as runtime: SubClassA

Note over Ref,Obj: Same instance, shown as base-type reference + runtime subtype
Client->>Ref: templateMethod()
activate Ref
Ref->>Obj: primitive1() (runtime binding)
Obj-->>Ref: overridden implementation

alt hook1() overridden
  Ref->>Obj: hook1() (runtime binding)
  Obj-->>Ref: custom hook logic
else hook1() not overridden
  Ref->>Ref: hook1() default no-op
end

Ref->>Obj: primitive2() (runtime binding)
Obj-->>Ref: overridden implementation
Ref-->>Client: done
deactivate Ref

```

#### Implemento `Activation_List`

Questa ActivationList è un bounded buffer da usare secondo il pattern Producer-Consumer per garantire la sincronizzazioen (in questo caso serializzazione). 
In questo caso il `Client_Thread` gioca il ruolo del producer inserendo `Method_Reqeuest` nella `Activation_List` attraverso i metodi sincronizzati del `MQ_Proxy`. Dall'altra parte lo scheduler thread gioca il ruolo del consumer prelevando col metodo `Activation_List::remove()` i `Method_Request` e richiamando il hook del `Servant` "bindato". 

```cpp
class Activation_List{
    public:
        enum { INFINITE = -1 };
        //define a trait
        typedef Activation_List_Iterator iterator;
        Activation_List(size_t high_water_mark);

        void insert(Method_Request* mr);
        void remove(Method_Request* mr);
    private:
        std::queue<Method_Request*> al_;
        std::mutex mtx_; //unused in this implementation, but needed for a thread-safe implementation
};

```


### Implemento lo `Scheduler`

Questo scheduler è un [Command Processor][Command Processor]

[Command Processor]: https://www.google.it
ciao

```cpp
class MQ_Scheduler {
public:
   // Initialize the <Activation_List> to have
   // the specified capacity and make <MQ_Scheduler>
   // run in its own thread of control.
   MQ_Scheduler (size_t high_water_mark);
 
   // … Other constructors/destructors, etc.
 
   // Put <Method_Request> into <Activation_List>. This
   // method runs in the thread of its client, i.e.
   // in the proxy’s thread.
   void insert (Method_Request *mr) {
      act_list_.insert (mr);
   }
 
   // Dispatch the method requests on their servant
   // in its scheduler’s thread of control.
   virtual void dispatch dispatch () {
       // Iterate continuously in a separate thread.
       for (;;) {
          Activation_List::iterator request;
          // The iterator’s <begin> method blocks
          // when the <Activation_List> is empty.
          for (request = act_list_.begin ();
             request != act_list_.end ();
             ++request) {
             // Select a method request whose
             // guard evaluates to true.
             if ((*request).can_run ()) {
                // Take <request> off the list.
                act_list_.remove (*request);
                (*request).call ();
                delete *request;
             }
             // Other scheduling activities can go here,
             // e.g., to handle when no <Method_Request>s
             // in the <Activation_List> have <can_run>
             // methods that evaluate to true.
          }
       }
    }
        // Entry point into the new thread.
    void *svc_run (void *args) {
        MQ_Scheduler *this_obj =
            static_cast<MQ_Scheduler *> (args);
        
        this_obj->dispatch ();
    }
    private:
    // List of pending Method_Requests.
    Activation_List act_list_;
};

```


Lo scheduler esegue in un thread di controllo diverso dai client. Ricordiamoci che il client thread usa il suo proxy per inserire i MethodRequest nel activationList. Inoltre qui si usa ThreadManager che non è altro che un Facde per la parte thread. 
Ultimo punto da determinare è il rendezvous del value (risultato dell'operazione) e return value policy. In sostanza dobbiamo capire come si restituisce il valore della computazione di un metodo invocato su un active object. 
1. *synchronous waiting*: block finché non c'è un valore
2. *Synchronous TIMED wait*: aspetta un intervallo finito di tempo
3. *Asincrono*: non c'è bisogno di spiegare

Il Future mi permette di avere un meccanismo a doppia viadi comunicazione. Evita disastri acqiusendo un lock e poi scrivendo il valore del risultato. 

```cpp
class Message_Future {
public:
   // Binds <this> and <pre> to the same <Msg._Future_Imp.>
   Message_Future (const Message_Future &f);
 
   // Initializes <Message_Future_Implementation> to
   // point to <message> m immediately.
   Message_Future (const Message &message);
 
   // Creates a <Msg._Future_Imp.>
   Message_Future ();
 
   // Binds <this> and <pre> to the same
   // <Msg._Future_Imp.>, which is created if necessary. Copy ctor
   void operator= (const Message_Future &f);
 
    //block indefinitely, to be refactored
   Message result () const;
private:
   // <Message_Future_Implementation> uses the Counted
   // Pointer idiom.
   Message_Future_Implementation *future_impl_;
};

```


Attenzione: col fatto che accedo al *body* del future solo attraverso il handle, io posso fare tutto il bookekeping di chi lo usa e quindo posso usare counter reference e usare il *Counted Pointer Idiom* che poi viene usato più spesso in C++11.

A questo punto un client che vuole usare tutto questo circo fa questo:

```cpp
MQ_Proxy message_queue;
 
// Obtain future and block thread until message arrives.
Message_Future future = message_queue.get ();
Message msg = future.result ();
 
// Transmit message to the consumer.
send (msg);

```


## Runnable Textbook Section Starts Here

From this point, run top-to-bottom. Dependencies come first, then classes, then user classes, then the runnable scenario.

## Active Object: Cohesive Textbook Walkthrough

In this section the flow is intentionally progressive: dependencies first, then infrastructure classes, then user-facing classes at the bottom that build the runnable example.

### 1) Dependencies

We start with standard C++ concurrency/runtime dependencies and a tiny logging helper used by all classes below.

In [1]:
#include <cstdio>
#include <deque>
#include <memory>
#include <queue>
#include <stdexcept>
#include <string>
#include <unordered_map>

void log_line(const std::string& s) {
  std::printf("%s\n", s.c_str());
}


`log_line()` was moved into the dependencies cell to keep setup simple and avoid duplicate definitions.

`log_line()` duplicate removed. Use the single definition in `textbook_dependencies_code`.

### 2) Domain Types

`Message` is the value moved by the Active Object. `SOCK_Stream` is a small demo output used to show where messages are sent.

In [2]:
struct Message {
  std::string address;
  std::string payload;

  size_t length() const { return payload.size(); }
};

class SOCK_Stream {
public:
  explicit SOCK_Stream(std::string consumer_name)
    : consumer_name_(std::move(consumer_name)) {}

  void send(const Message& msg, size_t len) {
    log_line("SOCK_Stream::send -> consumer=" + consumer_name_ +
             ", address=" + msg.address +
             ", bytes=" + std::to_string(len) +
             ", payload=\"" + msg.payload + "\"");
  }

private:
  std::string consumer_name_;
};



### 3) Servant (`MQ_Servant`)

The servant holds business state (here: bounded queue semantics) and provides operations invoked asynchronously by method requests.

In [3]:
class MQ_Servant {
public:
  explicit MQ_Servant(size_t mq_size) : mq_size_(mq_size) {}

  void put(const Message& msg) { mq_.push(msg); }

  Message get() {
    Message msg = mq_.front();
    mq_.pop();
    return msg;
  }

  bool is_full() const { return mq_.size() >= mq_size_; }
  bool is_empty() const { return mq_.empty(); }

private:
  std::queue<Message> mq_;
  size_t mq_size_;
};



### 4) Future Handle (`Message_Future`)

Notebook-safe simplified future: no thread primitives, just a shared ready/value state for didactic rendezvous.

In [4]:
class Message_Future_Implementation {
public:
  void set(const Message& message) {
    if (ready_) return;
    value_ = message;
    ready_ = true;
  }

  bool ready() const { return ready_; }

  Message result() const {
    if (!ready_) {
      throw std::runtime_error("Message_Future result() called before value is ready");
    }
    return value_;
  }

private:
  Message value_{};
  bool ready_ = false;
};

class Message_Future {
public:
  Message_Future()
    : future_impl_(std::make_shared<Message_Future_Implementation>()) {}

  Message_Future(const Message_Future&) = default;
  Message_Future& operator=(const Message_Future&) = default;

  explicit Message_Future(const Message& message) : Message_Future() {
    set(message);
  }

  void set(const Message& message) const {
    future_impl_->set(message);
  }

  bool ready() const {
    return future_impl_->ready();
  }

  Message result() const {
    return future_impl_->result();
  }

private:
  std::shared_ptr<Message_Future_Implementation> future_impl_;
};


### 5) Method Request Hierarchy

`Method_Request` is the command interface. `Put` and `Get` are concrete requests with guard logic (`can_run`).

In [5]:
class Method_Request {
public:
  virtual ~Method_Request() = default;
  virtual bool can_run() const = 0;
  virtual void call() = 0;
};

class Put : public Method_Request {
public:
  Put(MQ_Servant* rep, const Message& msg) : servant_(rep), msg_(msg) {}

  bool can_run() const override { return !servant_->is_full(); }

  void call() override {
    servant_->put(msg_);
    log_line("Put::call enqueued payload=\"" + msg_.payload + "\"");
  }

private:
  MQ_Servant* servant_;
  Message msg_;
};

class Get : public Method_Request {
public:
  Get(MQ_Servant* rep, Message_Future result)
    : servant_(rep), result_(std::move(result)) {}

  bool can_run() const override { return !servant_->is_empty(); }

  void call() override {
    result_.set(servant_->get());
    log_line("Get::call resolved Message_Future");
  }

private:
  MQ_Servant* servant_;
  Message_Future result_;
};


### 6) Activation List and Scheduler

Notebook-safe simplification: scheduler dispatch is explicit (single-thread), so the pattern is visible without runtime linker issues.

In [6]:
class Activation_List {
public:
  void insert(Method_Request* mr) {
    al_.emplace_back(mr);
  }

  bool empty() const { return al_.empty(); }

  std::unique_ptr<Method_Request> pop_front() {
    if (al_.empty()) return nullptr;
    auto mr = std::move(al_.front());
    al_.pop_front();
    return mr;
  }

  void push_back(std::unique_ptr<Method_Request> mr) {
    al_.push_back(std::move(mr));
  }

  std::size_t size() const { return al_.size(); }

private:
  std::deque<std::unique_ptr<Method_Request>> al_;
};

class MQ_Scheduler {
public:
  explicit MQ_Scheduler(std::size_t /*high_water_mark*/) {}

  void insert(Method_Request* mr) {
    act_list_.insert(mr);
    dispatch_ready();
  }

  void shutdown() {}

private:
  void dispatch_ready() {
    bool progressed = false;
    do {
      progressed = false;
      const std::size_t n = act_list_.size();
      for (std::size_t i = 0; i < n; ++i) {
        auto request = act_list_.pop_front();
        if (!request) break;

        if (request->can_run()) {
          request->call();
          progressed = true;
        } else {
          act_list_.push_back(std::move(request));
        }
      }
    } while (progressed && !act_list_.empty());
  }

  Activation_List act_list_;
};


### 7) Proxy (`MQ_Proxy`)

The proxy is the client-facing API. It creates concrete method requests and inserts them into the scheduler.

In [7]:
class MQ_Proxy {
public:
  enum { MQ_SIZE = 16 };

  explicit MQ_Proxy(std::size_t size = MQ_SIZE)
    : scheduler_(size), servant_(size) {}

  ~MQ_Proxy() { scheduler_.shutdown(); }

  void put(const Message& msg) {
    Method_Request* mr = new Put(&servant_, msg);
    scheduler_.insert(mr);
  }

  Message_Future get() {
    Message_Future result;
    Method_Request* mr = new Get(&servant_, result);
    scheduler_.insert(mr);
    return result;
  }

  bool is_empty() const {
    return servant_.is_empty();
  }

private:
  MQ_Scheduler scheduler_;
  MQ_Servant servant_;
};


### 8) User Classes and Full Runnable Example

User classes stay close to the message-router leitmotif, with explicit `drain_*` steps for notebook execution.

In [8]:
class Consumer_Handler {
public:
  explicit Consumer_Handler(std::string consumer_name)
    : consumer_name_(std::move(consumer_name)),
      msg_q_(32),
      connection_(consumer_name_) {}

  void put(const Message& msg) { msg_q_.put(msg); }

  void drain_one() {
    if (msg_q_.is_empty()) return;
    Message msg = msg_q_.get().result();
    connection_.send(msg, msg.length());
  }

  void drain_all() {
    while (!msg_q_.is_empty()) {
      drain_one();
    }
    log_line("Consumer_Handler::drain_all completed for " + consumer_name_);
  }

  void stop() {}

private:
  std::string consumer_name_;
  MQ_Proxy msg_q_;
  SOCK_Stream connection_;
};

class Supplier_Handler {
public:
  void bind_route(const std::string& address, Consumer_Handler& consumer) {
    routing_table_[address] = &consumer;
  }

  void route_message(const Message& msg) {
    auto it = routing_table_.find(msg.address);
    if (it == routing_table_.end()) {
      log_line("Supplier_Handler::route_message no route for address=" + msg.address);
      return;
    }

    Consumer_Handler* consumer_handler = it->second;
    consumer_handler->put(msg);
  }

private:
  std::unordered_map<std::string, Consumer_Handler*> routing_table_;
};


In [9]:
// Showcase 1/4: setup (notebook-safe, explicit execution).
#ifndef TEXTBOOK_SHOWCASE_STATE
#define TEXTBOOK_SHOWCASE_STATE
Consumer_Handler* billing_demo = nullptr;
Consumer_Handler* support_demo = nullptr;
Supplier_Handler* supplier_demo = nullptr;
#endif

if (billing_demo) { delete billing_demo; billing_demo = nullptr; }
if (support_demo) { delete support_demo; support_demo = nullptr; }
if (supplier_demo) { delete supplier_demo; supplier_demo = nullptr; }

billing_demo = new Consumer_Handler("billing");
support_demo = new Consumer_Handler("support");
supplier_demo = new Supplier_Handler();

supplier_demo->bind_route("billing", *billing_demo);
supplier_demo->bind_route("support", *support_demo);

log_line("showcase setup complete");


showcase setup complete


In [10]:
// Showcase 2/4: first routed batch + explicit drain.
if (!supplier_demo || !billing_demo || !support_demo) {
  log_line("showcase error: run setup cell first");
} else {
  supplier_demo->route_message({"billing", "invoice #A-100"});
  supplier_demo->route_message({"support", "reset password"});
  billing_demo->drain_all();
  support_demo->drain_all();
  log_line("showcase batch 1 routed and drained");
}


Put::call enqueued payload="invoice #A-100"
Put::call enqueued payload="reset password"
Get::call resolved Message_Future
SOCK_Stream::send -> consumer=billing, address=billing, bytes=14, payload="invoice #A-100"
Consumer_Handler::drain_all completed for billing
Get::call resolved Message_Future
SOCK_Stream::send -> consumer=support, address=support, bytes=14, payload="reset password"
Consumer_Handler::drain_all completed for support
showcase batch 1 routed and drained


In [11]:
// Showcase 3/4: second routed batch + explicit drain (no sleep needed).
if (!supplier_demo || !billing_demo || !support_demo) {
  log_line("showcase error: run setup cell first");
} else {
  supplier_demo->route_message({"support", "vpn down"});
  supplier_demo->route_message({"billing", "payment confirmed"});
  support_demo->drain_all();
  billing_demo->drain_all();
  log_line("showcase batch 2 routed and drained");
}


Put::call enqueued payload="vpn down"
Put::call enqueued payload="payment confirmed"
Get::call resolved Message_Future
SOCK_Stream::send -> consumer=support, address=support, bytes=8, payload="vpn down"
Consumer_Handler::drain_all completed for support
Get::call resolved Message_Future
SOCK_Stream::send -> consumer=billing, address=billing, bytes=17, payload="payment confirmed"
Consumer_Handler::drain_all completed for billing
showcase batch 2 routed and drained


In [12]:
// Showcase 4/4: cleanup.
if (billing_demo) { delete billing_demo; billing_demo = nullptr; }
if (support_demo) { delete support_demo; support_demo = nullptr; }
if (supplier_demo) { delete supplier_demo; supplier_demo = nullptr; }

log_line("showcase cleanup complete");


showcase cleanup complete
